# Quantum state tomography (QST) with LS, CS and CVX

In [4]:
# To call the libraries for the GD-QST
import sys
sys.path.insert(0, '..')
# You have to change the path of the library 
import os

from qutip import * 
from itertools import *
import numpy as np
import matplotlib.pyplot as plt 
import qutip as qtp
#from qutip import basis, tensor
import jax.numpy as jnp
from gd_qst import least_square_qst, compressed_sensing_qst, cvx_qst

import time


### First we will define the number of qubits and the measurement operators. We will use the Pauli matrices.


In [5]:
number_qubits: int = 3
dimension: int = 2**number_qubits
HS: int = dimension

# Creating the Measurement operators (Pauli matrices)

# computation basis, important for the other methods
def qubit_computation_basis(n):
    basis_states = [basis(2, i) for i in range(2)]
    return [tensor(*state) for state in product(basis_states, repeat=n)]

computation_basis = qubit_computation_basis(number_qubits)

operator_basis = [Qobj(tensor(computation_basis[i],computation_basis[j].dag()), 
                dims=([[2**number_qubits], [2**number_qubits]])) for i in range(len(computation_basis)) for j in range(len(computation_basis))]

pauli = [qeye(2), sigmax(), sigmay(), sigmaz()]
pauli_sys = [tensor(*op) for op in product(pauli, repeat=number_qubits)] # using itertools

Measu_ope = [Qobj(pauli_sys[i], dims=([[2**number_qubits], [2**number_qubits]])) for i in range(len(pauli_sys))]

print(Measu_ope[1:3])


#-----------------------------------------------------------------------
# It is necesary to transform the measurement operators from a numpy 
# type to a jax type (jax.np), that because of the jax grad 

ops_np = [op.full() for op in Measu_ope] # transforming from qutip object to numpy
ops_jnp = jnp.asarray(ops_np) # transforming from numpy to jax.np

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


[Quantum object: dims=[[8], [8]], shape=(8, 8), type='oper', isherm=True
Qobj data =
[[0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0.]], Quantum object: dims=[[8], [8]], shape=(8, 8), type='oper', isherm=True
Qobj data =
[[0.+0.j 0.-1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.-1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.-1.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+1.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.-1.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+1.j 0.+0.j]]]


## Implementing LS, CS, and CVX

For the case of a random density matrix for 3 qubits as the original density matrix (the one we want to reconstruct) created with QuTip $\rho_{random}$

In [6]:

max_itera: int = 300
batch_s = round(0.5*len(ops_jnp)) #len(ops_jnp)=4**N

# creating the original densirty matrix, this with a random rho from QuTip
rho_or = rand_dm(2**number_qubits)
data = qtp.expect(Measu_ope,rho_or)


#LS-QST
rho_ls, f_ls, t_ls = least_square_qst(measurement_ops=Measu_ope, basis_set=operator_basis, B=data, dimension=2**number_qubits, rho_ideal=rho_or)

#CS-QST
rho_cs, f_cs, t_cs = compressed_sensing_qst(measurement_ops=Measu_ope, basis_set=operator_basis, B=data, gamma=0, dimension=2**number_qubits, rho_ideal=rho_or)

rho_cvx, f_cvx, t_cvx = cvx_qst(measurement_ops=Measu_ope, basis_set=operator_basis, B=data, gamma=0, dimension=2**number_qubits, rho_ideal=rho_or)



In the cases of LS, CS, and CVX we just have a float value with the value of fidelity, the total time of reconstruction and the reconstructed density matrix. 

In [7]:
print('Final fidelity (LS) and average time in seconds :', f_ls, t_ls)
print('Final fidelity (CS) and average time in seconds :', f_cs, t_cs)
print('Final fidelity (CVX) and average time in seconds :', f_cvx, t_cvx)


Final fidelity (LS) and average time in seconds : 0.9999999999999749 0.04627203941345215
Final fidelity (CS) and average time in seconds : 0.9999999994975198 0.0576324462890625
Final fidelity (CVX) and average time in seconds : 0.9999999999963989 0.04280400276184082


In [8]:
print(rho_ls)

Quantum object: dims=[[8], [8]], shape=(8, 8), type='oper', isherm=True
Qobj data =
[[ 0.20826179+0.j         -0.00361344-0.01841851j  0.05449776+0.01503864j
   0.02256158-0.12491749j  0.00279598-0.07048509j  0.05409679-0.05308608j
   0.06354944+0.00823584j -0.03194077+0.01155733j]
 [-0.00361344+0.01841851j  0.05648059+0.j         -0.00070565+0.00739087j
   0.01571412-0.00164342j -0.01287864+0.02029771j  0.00504956-0.01769053j
   0.01506504+0.04204345j -0.00473074+0.01919361j]
 [ 0.05449776-0.01503864j -0.00070565-0.00739087j  0.12201116+0.j
  -0.04757943-0.0356821j  -0.02316136+0.01440593j  0.01740882-0.08258285j
   0.0134008 -0.00396079j  0.01611786+0.03024247j]
 [ 0.02256158+0.12491749j  0.01571412+0.00164342j -0.04757943+0.0356821j
   0.13924982+0.j          0.0694554 -0.01177087j  0.02645476+0.07715852j
   0.01197581+0.04625333j -0.05708889-0.01611221j]
 [ 0.00279598+0.07048509j -0.01287864-0.02029771j -0.02316136-0.01440593j
   0.0694554 +0.01177087j  0.12248613+0.j         -0.03